# using the varies of plots


In [1]:
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import os
import plotly.express as px

In [4]:
JPM_df = yf.Ticker("JPM").history(start=datetime(2020, 11, 30), end=datetime(2025, 11, 30))
AAPL_df = yf.Ticker("AAPL").history(start=datetime(2020, 11, 30), end=datetime(2025, 11, 30))
META_df = yf.Ticker("META").history(start=datetime(2020, 11, 30), end=datetime(2025, 11, 30))
KO_df = yf.Ticker("KO").history(start=datetime(2020, 11, 30), end=datetime(2025, 11, 30))
SP500_df = yf.Ticker("^GSPC").history(start=datetime(2020, 11, 30), end=datetime(2025, 11, 30))

In [ ]:
PriceDict = {
    "JPM" : JPM_df.reset_index(),
    "AAPL": AAPL_df.reset_index(),
    "META": META_df.reset_index(),
    "KO": KO_df.reset_index(),
    "S&P500": SP500_df.reset_index()
}

In [3]:


def save_dict(dict, folder_name):
    """
    Save a dictionary of DataFrames into CSV files inside a folder.
    
    data_dict: dict like {"AAPL": df, "JPM": df2}
    folder_name: str name of folder to save files
    """
    os.makedirs(folder_name, exist_ok=True)

    for k, v in dict.items():
        if hasattr(v, "to_csv"):
            file_path = os.path.join(folder_name, f"{k}.csv")
            v.to_csv(file_path, index=False)
        else:
            print(f"Skipping {k}: value is not a DataFrame")
    print("save complete")


def load_dict(folder_name):
    """
    Loads all CSV files from a folder into a dictionary of DataFrames.
    Returns: {"filename_without_extension": dataframe}
    """

    if not os.path.isdir(folder_name):
        raise FileNotFoundError(f"Folder '{folder_name}' does not exist.")

    loaded_dict = {}

    for file in os.listdir(folder_name):
        if file.endswith(".csv"):
            key = file[:-4]  # remove ".csv"
            file_path = os.path.join(folder_name, file)
            loaded_dict[key] = pd.read_csv(file_path)

    print(f"Loaded {len(loaded_dict)} DataFrames from '{folder_name}'.")
    return loaded_dict

In [4]:
PriceDict = load_dict("price_of_stocks")

Loaded 5 DataFrames from 'price_of_stocks'.


In [42]:
PriceDict = {
    "JPM" : JPM_df.reset_index(),
    "AAPL": AAPL_df.reset_index(),
    "META": META_df.reset_index(),
    "KO": KO_df.reset_index(),
    "S&P500": SP500_df.reset_index()
}


In [9]:
# I only care about the average price: (high + low ) / 2, I want to create a table with prices of stocks in dictionary
def extract_stocks_historical_price_from_dict(dict):
    store = pd.DataFrame({})
    isFirst = True
    for key, value in dict.items():
        if(isFirst):
            store["Date"] = value["Date"]
            isFirst = False
        store[key] = (value["High"] + value["Low"]) / 2
    return store


In [10]:
portfolio_stocks_return = extract_stocks_historical_price_from_dict(PriceDict)

In [11]:
portfolio_stocks_return

,Date,AAPL,JPM,KO,META,S&P500
0,2020-11-30 00:00:00-05:00,115.733748,104.729767,44.736098,272.674042,3614.284912
1,2020-12-01 00:00:00-05:00,118.508099,105.924206,45.104319,282.389110,3662.160034
2,2020-12-02 00:00:00-05:00,118.887727,106.086702,44.952687,284.550810,3657.900024
3,2020-12-03 00:00:00-05:00,119.729768,106.380919,45.416208,282.120818,3669.949951
4,2020-12-04 00:00:00-05:00,118.946123,107.579751,46.191621,279.656013,3685.069946
...,...,...,...,...,...,...
1251,2025-11-21 00:00:00-05:00,269.500000,297.244995,72.340000,589.989990,6590.984863
1252,2025-11-24 00:00:00-05:00,273.949997,297.165009,72.365002,607.165009,6673.225098
1253,2025-11-25 00:00:00-05:00,277.815002,300.029999,72.535000,627.674988,6718.189941
1254,2025-11-26 00:00:00-05:00,278.080002,305.865005,72.739998,634.994995,6807.655029


In [12]:
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    for i in raw_prices_df.columns[1:]:
          scaled_prices_df[i] = raw_prices_df[i]/raw_prices_df[i][0]
    return scaled_prices_df
def plot_financial_data(df, title):
    
    fig = px.line(title = title)
    
    # For loop that plots all stock prices in the pandas dataframe df
    # Note that index starts with 1 because we want to skip the date column
    
    for i in df.columns[1:]:
        fig.add_scatter(x = df['Date'], y = df[i], name = i)
        fig.update_traces(line_width = 5)
        fig.update_layout({'plot_bgcolor': "white"})

    fig.show()

In [13]:
# First lets look about dayily return rate:
daily_returns_df = portfolio_stocks_return.iloc[:, 1:].pct_change() * 100
daily_returns_df.replace(np.nan, 0, inplace=True)
daily_returns_df.insert(0, "Date", portfolio_stocks_return["Date"])

In [49]:
daily_returns_df

,Date,JPM,AAPL,META,KO,S&P500
0,2020-11-30 00:00:00-05:00,0.000000,0.000000,0.000000,0.000000,0.000000
1,2020-12-01 00:00:00-05:00,1.140496,2.397184,3.562887,0.823096,1.324608
2,2020-12-02 00:00:00-05:00,0.153408,0.320339,0.765504,-0.336180,-0.116325
3,2020-12-03 00:00:00-05:00,0.277336,0.708265,-0.853975,1.031131,0.329422
4,2020-12-04 00:00:00-05:00,1.126925,-0.654511,-0.873670,1.707349,0.411995
...,...,...,...,...,...,...
1251,2025-11-21 00:00:00-05:00,-2.234908,-0.434101,-0.847846,1.822786,-0.920223
1252,2025-11-24 00:00:00-05:00,-0.026909,1.651205,2.911069,0.034561,1.247769
1253,2025-11-25 00:00:00-05:00,0.964108,1.410843,3.377991,0.234918,0.673810
1254,2025-11-26 00:00:00-05:00,1.944808,0.095387,1.166210,0.282619,1.331684


In [50]:
plot_financial_data(daily_returns_df, "portfolio daily return")

In [21]:
# nope, not looking great beside Meta seem to have a very high beta, lets try to use
a = price_scaling(portfolio_stocks_return)
plot_financial_data(a, "scaled price data")

In [22]:
# Hum, jpm seem perform amazing, Meta didnt do good recently, KO underperform the market.

In [19]:
def price_scaling(raw_prices_df):
    scaled_prices_df = raw_prices_df.copy()
    for i in raw_prices_df.columns[1:]:
          scaled_prices_df[i] = raw_prices_df[i]/raw_prices_df[i][0]
    return scaled_prices_df

price_scaling(portfolio_stocks_return)

,Date,AAPL,JPM,KO,META,S&P500
0,2020-11-30 00:00:00-05:00,1.000000,1.000000,1.000000,1.000000,1.000000
1,2020-12-01 00:00:00-05:00,1.023972,1.011405,1.008231,1.035629,1.013246
2,2020-12-02 00:00:00-05:00,1.027252,1.012957,1.004841,1.043557,1.012067
3,2020-12-03 00:00:00-05:00,1.034528,1.015766,1.015203,1.034645,1.015401
4,2020-12-04 00:00:00-05:00,1.027757,1.027213,1.032536,1.025606,1.019585
...,...,...,...,...,...,...
1251,2025-11-21 00:00:00-05:00,2.328621,2.838209,1.617039,2.163719,1.823593
1252,2025-11-24 00:00:00-05:00,2.367071,2.837446,1.617598,2.226706,1.846347
1253,2025-11-25 00:00:00-05:00,2.400467,2.864802,1.621398,2.301924,1.858788
1254,2025-11-26 00:00:00-05:00,2.402756,2.920516,1.625980,2.328770,1.883541


In [35]:
daily_returns_df

,JPM,AAPL,META,KO,S&P500
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,1.140496,2.397184,3.562887,0.823096,1.324608
2,0.153408,0.320339,0.765504,-0.336180,-0.116325
3,0.277336,0.708265,-0.853975,1.031131,0.329422
4,1.126925,-0.654511,-0.873670,1.707349,0.411995
...,...,...,...,...,...
1251,-2.234908,-0.434101,-0.847846,1.822786,-0.920223
1252,-0.026909,1.651205,2.911069,0.034561,1.247769
1253,0.964108,1.410843,3.377991,0.234918,0.673810
1254,1.944808,0.095387,1.166210,0.282619,1.331684
